In [1]:
import pandas as pd
from utils.connect_to_db import db_connect
import json
import pickle

In [11]:
resume_engine = db_connect("corpus2_db")

batch_size = 1000
offset = 0
raw_df = pd.DataFrame()
count = 0

In [12]:
while True:
    query = f""" SELECT resource_id, curriculum_json FROM curriculum_data_tmp_phrase where curriculum_json is not null 
    LIMIT {batch_size} OFFSET {offset}"""
    batch_df = pd.read_sql(query, con=resume_engine)

    if batch_df.empty:
        break

    offset += batch_size
    raw_df = pd.concat([raw_df, batch_df], ignore_index=True)
    count += 1
    print(f"Records Count : {count * batch_size}")
    

Records Count : 1000
Records Count : 2000
Records Count : 3000
Records Count : 4000
Records Count : 5000
Records Count : 6000
Records Count : 7000
Records Count : 8000
Records Count : 9000
Records Count : 10000
Records Count : 11000
Records Count : 12000
Records Count : 13000
Records Count : 14000
Records Count : 15000
Records Count : 16000
Records Count : 17000
Records Count : 18000
Records Count : 19000
Records Count : 20000
Records Count : 21000
Records Count : 22000
Records Count : 23000
Records Count : 24000
Records Count : 25000
Records Count : 26000
Records Count : 27000
Records Count : 28000
Records Count : 29000
Records Count : 30000
Records Count : 31000
Records Count : 32000
Records Count : 33000
Records Count : 34000
Records Count : 35000
Records Count : 36000
Records Count : 37000
Records Count : 38000
Records Count : 39000
Records Count : 40000
Records Count : 41000
Records Count : 42000
Records Count : 43000
Records Count : 44000
Records Count : 45000
Records Count : 460

In [16]:
len(raw_df)

311085

In [15]:
raw_df.to_csv("curriculum_data_tmp_phrase.csv", index=False)

In [17]:
batch_size = 1000
offset = 0
raw_df_2 = pd.DataFrame()
count = 0

In [18]:
while True:
    query_2 = f""" SELECT resource_id, curriculum_json FROM curriculum_data where curriculum_json is not null 
    LIMIT {batch_size} OFFSET {offset}"""
    batch_df = pd.read_sql(query_2, con=resume_engine)

    if batch_df.empty:
        break

    offset += batch_size
    raw_df_2 = pd.concat([raw_df_2, batch_df], ignore_index=True)
    count += 1
    print(f"Records Count : {count * batch_size}")
    

Records Count : 1000
Records Count : 2000
Records Count : 3000
Records Count : 4000
Records Count : 5000
Records Count : 6000
Records Count : 7000
Records Count : 8000
Records Count : 9000
Records Count : 10000
Records Count : 11000
Records Count : 12000
Records Count : 13000
Records Count : 14000
Records Count : 15000
Records Count : 16000
Records Count : 17000
Records Count : 18000
Records Count : 19000
Records Count : 20000
Records Count : 21000
Records Count : 22000
Records Count : 23000
Records Count : 24000
Records Count : 25000
Records Count : 26000
Records Count : 27000
Records Count : 28000
Records Count : 29000
Records Count : 30000
Records Count : 31000
Records Count : 32000
Records Count : 33000
Records Count : 34000
Records Count : 35000
Records Count : 36000
Records Count : 37000
Records Count : 38000
Records Count : 39000
Records Count : 40000
Records Count : 41000
Records Count : 42000
Records Count : 43000
Records Count : 44000
Records Count : 45000
Records Count : 460

In [19]:
resource_id_list = raw_df['resource_id'].tolist()
raw_df_2_filtered = raw_df_2[~raw_df_2['resource_id'].isin(resource_id_list)]

In [20]:
final_raw_df = pd.concat([raw_df, raw_df_2_filtered], ignore_index=True)

In [8]:
def get_raw_text_and_languages(text):
    """
    Extract raw text and resume probability from a JSON string.
    """
    try:
        data = json.loads(text)
        raw_text = data.get('rawText', '').strip()
        languages = data.get('languages', '')
        return raw_text, languages
    except:
        return "raw text not present", []
    

def process_batch(batch_df):
    """
    Process a single batch of data to extract raw text.
    """
    batch_df[['raw_text', 'languages']] = batch_df['curriculum_json'].apply(
        lambda x: pd.Series(get_raw_text_and_languages(x))
    )

    no_raw_text_parse_df = batch_df[batch_df['raw_text'] == "raw text not present"]
    print(f"Percentage of data for which parsable raw text is not present: {(len(no_raw_text_parse_df)/len(batch_df)) * 100:.2f}%")

    no_raw_text_df = batch_df[batch_df['raw_text'] == ""]
    print(f"Percentage of data for which raw text is not present: {(len(no_raw_text_df)/len(batch_df)) * 100:.2f}%")

    return batch_df[~batch_df['raw_text'].isin(['', "raw text not present"])]

def save_to_csv(df, writer):
    """
    Save processed data to the CSV file.
    """
    df[['resource_id', 'raw_text', 'languages']].to_csv(writer, index=False, header=False)

In [ ]:
final_raw_df = process_batch(final_raw_df)

In [28]:
len(final_raw_df)

454576

In [23]:
############## save the final dataframe to csv ##############
final_raw_df.to_csv("./data/extracted_data/curriculum_json_full.csv", index=False)

In [3]:
final_raw_df = pd.read_csv("./data/extracted_data/curriculum_json_full.csv")

In [4]:
len(final_raw_df)

454576

In [6]:
final_raw_df['resource_id'].iloc[454575]

5972774

In [ ]:
# output_path = parse_arguments()
import os
import csv
import gc

output_path = "./data/extracted_data/resume_details_cj_17_04_25.csv"
resume_engine = db_connect("corpus2_db")
batch_size = 1000
offset = 0

# Ensure the output directory exists
os.makedirs(os.path.dirname(output_path), exist_ok=True)

with open(output_path, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['resource_id', 'raw_text', 'languages'])
    # writer.writerow(['resource_id', 'raw_text'])

    chunk_size = 1000
    num_chunks = len(final_raw_df) // chunk_size + (len(final_raw_df) % chunk_size != 0)

    for i in range(num_chunks):
        batch_df = final_raw_df[i*chunk_size : min((i+1)*chunk_size, len(final_raw_df))]

        if batch_df.empty:
            break
        
        # output_folder = './data/json_files/'
        # save_json_to_files(batch_df, output_folder)
        
        processed_df = process_batch(batch_df)
        save_to_csv(processed_df, csv_file)
        
        # del batch_df, processed_df
        del processed_df
        gc.collect()

        offset += batch_size
        print(f"Processed {offset} records...")

In [10]:
output_raw_df = pd.read_csv(output_path)

PermissionError: [Errno 13] Permission denied

In [7]:
len(output_raw_df)

454576

In [ ]:
# resource_id_list = raw_df['resource_id'].tolist()

# query = f"SELECT * FROM curriculum_data where curriculum_json is not null"

# required_columns = ['resource_id', 'curriculum_name', 'curriculum_data', 'curriculum_json', 'updated_at']

# raw_df_2 = pd.read_sql(query, con=resume_engine)
# raw_df_2 = raw_df_2[required_columns]

# raw_df = raw_df[required_columns]

# raw_df_2_filtered = raw_df_2[~raw_df_2['resource_id'].isin(resource_id_list)]

# final_raw_df = pd.concat([raw_df, raw_df_2_filtered], ignore_index=True)

In [ ]:

with open('resume_data_dump.pkl', 'wb') as f:
    pickle.dump(final_raw_df, f)

with open('resume_data_dump.pkl', 'rb') as f:
    final_raw_df = pickle.load(f)


def get_json(x):
    try:
        return json.loads(x)
    except:
        return "Json Load Error"


def get_raw_text(x):
    try:
        return x['rawText']
    except:
        return "Raw Text Not Present"


def get_languages(x):
    try:
        return x['languages']
    except:
        return "Language Not Present"


final_raw_df['json_data'] = final_raw_df['curriculum_json'].apply(lambda x: get_json(x))
final_raw_df['raw_text'] = final_raw_df['json_data'].apply(lambda x: get_raw_text(x))
final_raw_df['languages'] = final_raw_df['json_data'].apply(lambda x: get_languages(x))

final_df_copy = final_raw_df[['resource_id', 'raw_text', 'languages']]

final_df_copy.to_excel('resume_data_dump.xlsx', index=False)